# Bike Sharing (MLFlow - KServe)

This notebook provides a detailed walkthrough of a comprehensive data science workflow, encompassing data preprocessing, model training and evaluation, hyperparameter tuning, experiment tracking via MLFlow, and model deployment using Seldon and KServe. The use case under consideration is the well-known bike sharing dataset, sourced from the UCI Machine Learning Repository.

![bike-sharing](images/bike-sharing.jpg)
(Photo by <a href="https://unsplash.com/@zaccastravels?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">ZACHARY STAINES</a> on <a href="https://unsplash.com/photos/KEhNcoCldbk?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>
  )

The dataset records the hourly and daily count of rental bikes between 2011 and 2012 in the Capital Bikeshare system, supplemented with corresponding weather and seasonal data. The primary objective of this dataset is to foster research into bike sharing systems, which are gaining significant attention due to their implications on traffic management, environmental sustainability, and public health.

The task associated with this dataset is regression, with 17,389 instances. The overarching goal is to construct a predictive model capable of forecasting bike rental demand. The primary target variable for prediction is the `cnt` attribute, representing the total count of rental bikes, inclusive of both casual and registered users.

By leveraging the other features in the dataset (such as date, season, year, month, hour, holiday, weekday, working day, weather conditions, temperature, perceived temperature, humidity, and wind speed), you can train a model to predict this count with high accuracy.

**References:**
- https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset
- https://docs.databricks.com/_static/notebooks/gbt-regression.html
- https://www.kaggle.com/pratsiuk/mlflow-experiment-automation-top-9
- https://mlflow.org/docs/latest/tracking.html

## Setting Up the Environment

The subsequent code cells are dedicated to importing the requisite dependencies and adjusting the logging level to `ERROR`. Additionally, it's recommended to establish a local directory for preserving the training artifacts generated during your experiment.

In [ ]:
import os
import json
import datetime
import itertools
import warnings

from urllib.parse import urlparse
from functools import partial

import git
import boto3
import kserve
import sklearn
import ipywidgets as widgets
import s3transfer
import requests
import graphviz
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn

from mlflow.models.signature import infer_signature
from mlflow import log_metric, log_param, log_artifact
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.inspection import permutation_importance
from pydotplus import graph_from_dot_data
from IPython.display import Image, display
from kubernetes import client, config
from kubernetes.client import V1ObjectMeta, V1Secret, V1ServiceAccount, V1ObjectReference, V1Container, V1EnvVar
from kserve import (V1beta1InferenceService, V1beta1InferenceServiceSpec, V1beta1SKLearnSpec,
                    V1beta1PredictorSpec, V1beta1ModelSpec, V1beta1ModelFormat)


plt.style.use("fivethirtyeight")
pd.plotting.register_matplotlib_converters()

warnings.filterwarnings('ignore')

In [ ]:
import logging

# Get all loggers
loggers = logging.Logger.manager.loggerDict.values()

# Iterate over all loggers and set their level to ERROR
# as we don't want to polute the output of the code cells
# with debugging messages.
for logger in loggers:
    if isinstance(logger, logging.Logger):
        logger.setLevel(logging.ERROR)

In [ ]:
if os.path.exists("model_artifacts"):
    os.system("rm -rf model_artifacts")
os.mkdir("model_artifacts")

## User Authentication Process

The subsequent step involves authenticating yourself with MLFlow. This is crucial for tracking your experiments and storing the training artifacts. Utilize your user credentials to establish a connection with the MLFlow service. Additionally, connect to the data fabric object store to preserve the metadata associated with your experiments. This ensures a comprehensive record of your work, facilitating future analysis and reproducibility.

In [ ]:
# Add heading
heading = widgets.HTML("<h2>MLflow Credentials</h2>")
display(heading)

ezaf_env_input = widgets.Text(description='EZAF Env:')
username_input = widgets.Text(description='Username:')
password_input = widgets.Password(description='Password:')
submit_button = widgets.Button(description='Submit')
success_message = widgets.Output()

ezaf_env = None
mlflow_username = None
mlflow_password = None

def submit_button_clicked(b):
    global ezaf_env, mlflow_username, mlflow_password
    ezaf_env = ezaf_env_input.value
    mlflow_username = username_input.value
    mlflow_password = password_input.value
    with success_message:
        success_message.clear_output()
        print("Credentials submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)

# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

# Display inputs and button
display(ezaf_env_input, username_input, password_input, submit_button, success_message)

In [ ]:
EZAF_ENV = ezaf_env
token_url = f"https://keycloak.{EZAF_ENV}.com/realms/UA/protocol/openid-connect/token"

data = {
    "username" : mlflow_username,
    "password" : mlflow_password,
    "grant_type" : "password",
    "client_id" : "ua-grant",
}

token_responce = requests.post(token_url, data=data, allow_redirects=True, verify=False)

token = token_responce.json()["access_token"]

In [ ]:
# Add heading
heading = widgets.HTML("<h2>Object Store Credentials</h2>")
display(heading)

# Access Key and Secret Key inputs
access_key_input = widgets.Text(description='Access Key:')
secret_key_input = widgets.Password(description='Secret Key:')
submit_button = widgets.Button(description='Submit')
success_message = widgets.Output()

minio_access_key = None
minio_secret_key = None

def submit_button_clicked(b):
    global minio_access_key, minio_secret_key
    minio_access_key = access_key_input.value
    minio_secret_key = secret_key_input.value
    with success_message:
        success_message.clear_output()
        print("Credentials submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)

# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

# Display inputs and button
display(access_key_input, secret_key_input, submit_button, success_message)

In [ ]:
def set_experiment(exp_name):
    """Register an experiment in MLFlow.
    
    args:
      exp_name (str): The name of the experiment.
    """
    try:
        usr = "hpedemo-user01"
        os.environ['MLFLOW_TRACKING_INSECURE_TLS'] = "true"
        os.environ["AWS_ACCESS_KEY_ID"] = minio_access_key
        os.environ["AWS_SECRET_ACCESS_KEY"] = minio_secret_key
        os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"https://home.{EZAF_ENV}.com:31900"
        os.environ['MLFLOW_S3_IGNORE_TLS'] = "true"
        os.environ['MLFLOW_TRACKING_TOKEN'] = token
        mlflow.set_tracking_uri(f"https://mlflow.{EZAF_ENV}.com")
        mlflow.set_experiment(exp_name)
        mlflow.set_tag('mlflow.user', usr)
    except Exception as e:
        raise RuntimeError(f"Failed to set the experiment: {e}")

In [ ]:
# Set up an experiment with set_exp from ezmllib.mlflow
experiment_name = 'bike-sharing-exp-07'
set_experiment(experiment_name)

## Load the Dataset

With the preliminary setup complete, we can now proceed to load the dataset. The data is provided in a CSV format, which can be conveniently loaded using the Pandas library in Python. To get a glimpse of the dataset, we'll display the first five rows using the `head()` method of the DataFrame. This initial exploration will provide a snapshot of the data we'll be working with.

In [ ]:
# Dataset is already available in github repository if not you can download and extract csv files as well.
#!wget -e use_proxy=yes -e http_proxy=http://web-proxy.corp.hpecorp.net:8080 -nc "http://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip"
#!unzip -o "Bike-Sharing-Dataset.zip"
#!rm -rf "Bike-Sharing-Dataset.zip"

In [ ]:
# Load input data into pandas dataframe
bike_sharing = pd.read_csv("bike-sharing.csv")
bike_sharing.head()   

## Data preprocessing

In this phase, we will prepare our data for the subsequent stages of our analysis. This involves cleaning, transforming, and structuring the data to ensure it is in the optimal format for our machine learning model.

In [ ]:
# Remove unused columns
bike_sharing.drop(columns=["instant", "dteday", "registered", "casual"], inplace=True)

# Use better names
bike_sharing.rename(
    columns={
        "yr": "year",
        "mnth": "month",
        "hr": "hour_of_day",
        "holiday": "is_holiday",
        "workingday": "is_workingday",
        "weathersit": "weather_situation",
        "temp": "temperature",
        "atemp": "feels_like_temperature",
        "hum": "humidity",
        "cnt": "rented_bikes",
    }, inplace=True)

# Convert every data point to `float64`
cols = bike_sharing.select_dtypes(exclude=['float64']).columns
for i in ['season', 'year', 'month', 'hour_of_day', 'is_holiday',
          'weekday', 'is_workingday', 'weather_situation', 'rented_bikes']:
    bike_sharing[i] = bike_sharing[i].astype('float64')

## Data Visualization

In this section, we will employ various visualization techniques to better understand our data. By creating graphical representations of the data, we can identify patterns, trends, and correlations that might not be evident from the raw data alone. This step is crucial in guiding our subsequent analysis and model building process.

In [ ]:
hour_of_day_agg = bike_sharing.groupby(["hour_of_day"])["rented_bikes"].sum()

hour_of_day_agg.plot(
    kind="line", 
    title="Total rented bikes by hour of day",
    xticks=hour_of_day_agg.index,
    figsize=(10, 5),
)

plt.show()

## Prepare training and test data sets

In this section, we will partition our data into training and test datasets. This is a crucial step in the machine learning workflow, allowing us to train our model on a subset of the data (the training set), and then evaluate its performance on unseen data (the test set). This process helps ensure that our model generalizes well to new data and is not simply memorizing the training data.

In [ ]:
# Split the dataset randomly into 70% for training and 30% for testing.
X = bike_sharing.drop("rented_bikes", axis=1)
y = bike_sharing.rented_bikes
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

print(f"Training samples: {X_train.size}")
print(f"Test samples: {X_test.size}")

## Establishing Evaluation Metrics

Before proceeding to the training stage, we will define the evaluation metrics that will be used to assess the performance of our model. These metrics will provide quantitative measures of the model's accuracy, helping us understand how well our model is performing and where improvements can be made. This step is crucial in ensuring that our model meets the desired performance standards.

### Root Mean Square Error (RMSE)

One of the evaluation metrics we will use is the Root Mean Square Error (RMSE). This metric provides a measure of the differences between the values predicted by our model and the actual values. By taking the square root of the average of these squared differences, RMSE gives us a sense of the magnitude of the prediction errors. Lower RMSE values indicate a better fit of the model to the data.

References: 
- https://medium.com/@xaviergeerinck/artificial-intelligence-how-to-measure-performance-accuracy-precision-recall-f1-roc-rmse-611d10e4caac
- https://www.kaggle.com/residentmario/model-fit-metrics#Root-mean-squared-error-(RMSE)

In [ ]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def rmse_score(y, y_pred):
    score = rmse(y, y_pred)
    message = "RMSE score: {:.4f}".format(score)
    return score, message

### Cross-Validation RMSLE score

Another evaluation metric we will employ is the Root Mean Squared Logarithmic Error (RMSLE) score, calculated through cross-validation. Cross-validation is a robust technique that averages measures of prediction accuracy to derive a more precise estimate of model performance.

The RMSLE score is especially valuable in your situation as it penalizes underestimates more than overestimates. Therefore, it is an essential metric for our bike sharing demand prediction model, ensuring that we avoid scenarios where the available number of bikes falls short of the demand.

References: 
- https://en.wikipedia.org/wiki/Cross-validation_(statistics)
- https://www.kaggle.com/carlolepelaars/understanding-the-metric-rmsle


In [ ]:
def rmsle_clip(estimator, x, y):
    """Clip negative prediction numbers before calculating RMSLE."""
    y_pred = estimator.predict(x)
    y_pred_clipped = np.clip(y_pred, a_min=0, a_max=None)
    return sklearn.metrics.mean_squared_log_error(y, y_pred_clipped, squared=False)

def rmsle_cv(model, X_train, y_train):
    kf = KFold(n_splits=4, shuffle=True, random_state=42).get_n_splits(X_train.values)
    # Evaluate RMSLE score by cross-validation
    rmsle = cross_val_score(model, X_train.values, y_train, scoring=rmsle_clip, cv=kf, error_score="raise")
    return rmsle

def rmsle_cv_score(model, X_train, y_train):
    score = rmsle_cv(model, X_train, y_train)
    message = "Cross-Validation RMSLE score: {:.4f} (std = {:.4f})".format(score.mean(), score.std())
    return score, message

## Feature Importance

In this section, we will analyze the importance of each feature in our dataset. Feature importance refers to techniques that assign a score to input features based on how useful they are at predicting a target variable.

Understanding which features are most influential in predicting the target variable can provide valuable insights into our dataset and the underlying model. This can help us interpret the model's predictions, and can guide further data collection and feature engineering efforts.

References:
- https://medium.com/bigdatarepublic/feature-importance-whats-in-a-name-79532e59eea3

In [ ]:
def model_feature_importance(model):
    feature_importance = pd.DataFrame(
        model.feature_importances_,
        index=X_train.columns,
        columns=["Importance"])

    # sort by importance
    feature_importance.sort_values(by="Importance", ascending=False, inplace=True)

    # plot
    plt.figure(figsize=(10, 4))
    sns.barplot(
        data=feature_importance.reset_index(),
        y="index",
        x="Importance",
    ).set_title("Feature Importance")

    # save image
    plt.savefig("model_artifacts/feature_importance.png", bbox_inches='tight')

## Permutation Importance

Permutation Importance is a technique used to measure feature importance. It works by randomly shuffling a single feature in the validation data and measuring the decrease in the model's performance. The features that cause the most significant drop in performance are considered the most important.

This method provides a straightforward way to interpret the influence of each feature on the model's predictions. It can help us understand which features are driving the model's decisions and where we might focus our attention for further data analysis or feature engineering.

References:
- https://www.kaggle.com/dansbecker/permutation-importance

In [ ]:
def model_permutation_importance(model):
    p_importance = permutation_importance(model, X_test, y_test, random_state=42, n_jobs=-1)

    # sort by importance
    sorted_idx = p_importance.importances_mean.argsort()[::-1]
    p_importance = pd.DataFrame(
        data=p_importance.importances[sorted_idx].T,
        columns=X_train.columns[sorted_idx]
    )

    # plot
    plt.figure(figsize=(10, 4))
    sns.barplot(
        data=p_importance,
        orient="h"
    ).set_title("Permutation Importance")

    # save image
    plt.savefig("model_artifacts/permutation_importance.png", bbox_inches="tight")

## Decision Tree Visualization

In this section, we will visualize the decision tree model. A decision tree is a flowchart-like structure where each internal node represents a feature (or attribute), each branch represents a decision rule, and each leaf node represents an outcome.

Visualizing the decision tree provides a clear understanding of how the model makes predictions. It allows us to see the decision paths and rules the model uses, making it one of the most interpretable machine learning models. This can be particularly useful when explaining the model's decisions to stakeholders.

References:
- https://towardsdatascience.com/visualizing-decision-trees-with-python-scikit-learn-graphviz-matplotlib-1c50b4aa68dc 

In [ ]:
def model_tree_visualization(model):
    # generate visualization
    tree_dot_data = tree.export_graphviz(
        # Get the first tree
        # TODO: Visualize every tree
        decision_tree=model.estimators_[0, 0],
        label="all",
        feature_names=X_train.columns,
        filled=True,
        rounded=True,
        proportion=True,
        impurity=False,
        precision=1,
    )

    # save image
    graph_from_dot_data(tree_dot_data).write_png("model_artifacts/Decision_Tree_Visualization.png")

    # show tree
    return graphviz.Source(tree_dot_data)

## MLflow Tracking

In this phase, we will utilize MLflow Tracking, a component of MLflow that logs and tracks experiment data. This includes parameters, metrics, and artifacts of machine learning models during the training process.

MLflow Tracking provides a centralized repository for metadata associated with your experiments, making it easier to compare different runs, reproduce results, and share findings with your team. This is a crucial step in maintaining an organized and efficient machine learning workflow.

First, let's setup the logger.

References:
- https://www.mlflow.org/docs/latest/cli.html#mlflow-ui

In [ ]:
# Track params and metrics
def log_mlflow_run(model, signature):
    # Auto-logging for scikit-learn estimators
    # mlflow.sklearn.autolog()

    # log estimator_name name
    name = model.__class__.__name__
    mlflow.set_tag("estimator_name", name)

    # log input features
    mlflow.set_tag("features", str(X_train.columns.values.tolist()))

    # Log tracked parameters only
    mlflow.log_params({key: model.get_params()[key] for key in parameters})

    mlflow.log_metrics({
        'RMSLE_CV': score_cv.mean(),
        'RMSE': score})

    # log training loss
    for s in model.train_score_:
        mlflow.log_metric("Train Loss", s)

    # Save model to artifacts
    mlflow.sklearn.log_model(model, "model")#, signature=signature)

    # log charts
    mlflow.log_artifacts("model_artifacts")

    # misc
    # Log all model parameters
    # mlflow.log_params(model.get_params())
    mlflow.log_param("Training size", X_test.size) 
    mlflow.log_param("Test size", y_test.size)

## Model Training and Hyperparameter Tuning

In this section, we will focus on training our model and tuning its hyperparameters. For this particular use case, we will employ the following approach:

- Approach: We will use a Supervised Learning method, specifically a Decision Tree model. Decision Trees are intuitive and easy-to-interpret models that make decisions based on a set of rules inferred from the features.
- Tree Type: Given that our task is to predict a continuous target variable (the count of total rental bikes), we will use a Regression Tree. Regression Trees predict outcomes by learning simple decision rules inferred from the features.
- Technique/Ensemble Method: To improve the performance of our Decision Tree model, we will use an ensemble method known as Gradient Boosting. Gradient Boosting combines several weak learners (in this case, Decision Trees) to create a robust predictive model. It trains models in a gradual, additive, and sequential manner, with each new model correcting the errors made by the previous ones.

By carefully tuning the hyperparameters of our Gradient Boosting model, we can optimize its performance and ensure it generalizes well to new data.

References:
- GBRT (Gradient Boosted Regression Tree): https://orbi.uliege.be/bitstream/2268/163521/1/slides.pdf
- Choosing a model: https://scikit-learn.org/stable/tutorial/machine_learning_map
- Machine Learning Models Explained
: https://docs.paperspace.com/machine-learning/wiki/machine-learning-models-explained
- Gradient Boosted Regression Trees: https://orbi.uliege.be/bitstream/2268/163521/1/slides.pdf


In [ ]:
# GBRT (Gradient Boosted Regression Tree) scikit-learn implementation 
model_class = GradientBoostingRegressor

Set the training's process hyperparameters.

In [ ]:
parameters = {
    "learning_rate": [0.1, 0.05, 0.01],
    "max_depth": [4, 5, 6],
    # "verbose": True,
}

To optimize the performance of our model, we will tune its hyperparameters using a method known as Grid Search.

Grid Search is a traditional method for hyperparameter tuning. It works by defining a grid of hyperparameters and then evaluating the model performance for each point on the grid. You can think of this as an exhaustive search through a manually specified subset of the hyperparameter space of the chosen algorithm.

By using Grid Search, we can systematically work through multiple combinations of hyperparameters to determine the optimal values that improve the performance of our model. This process can significantly enhance the predictive accuracy of our model.

References:
- More advanced tuning techniques: https://research.fb.com/efficient-tuning-of-online-systems-using-bayesian-optimization/

In [ ]:
# generate parameters combinations
params_keys = parameters.keys()
params_values = [
    parameters[key] if isinstance(parameters[key], list) else [parameters[key]]
    for key in params_keys]

runs_parameters = [
    dict(zip(params_keys, combination)) for combination in itertools.product(*params_values)]

## Model Training

Now that we have prepared our data and set up our model, the next step is to train the model. During this process, the model will learn from the features of our training data to predict the target variable.

Model training involves adjusting the model to minimize the difference between the predicted and actual values, a process guided by a specific learning algorithm. In our case, we are using a Gradient Boosting model, which will learn to correct its errors in a gradual, additive, and sequential manner.

This is a crucial step in our machine learning workflow, as the quality of the model's predictions heavily depends on the effectiveness of the training process.

In [ ]:
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=len(runs_parameters),
    # description='Training progress',
    bar_style='info',
    orientation='horizontal'
)

progress_text = widgets.Output()

display(progress_bar, progress_text)

# training loop
for i, run_parameters in enumerate(runs_parameters):
    progress_bar.description = f"Run {i+1}/{len(runs_parameters)}"
    # mlflow: stop active runs if any
    if mlflow.active_run():
        mlflow.end_run()
    # mlflow:track run
    mlflow.start_run(run_name=f"Run {i}")

    # create model instance
    model = model_class(**run_parameters)

    # train
    model.fit(X_train, y_train)

    # get evaluations scores
    ypred = model.predict(X_test)
    score, message = rmse_score(y_test, model.predict(X_test))
    score_cv, message_cv = rmsle_cv_score(model, X_train, y_train)
    
    # generate charts
    model_feature_importance(model)
    plt.close()
    model_permutation_importance(model)
    plt.close()
    # model_tree_visualization(model)

    # get model signature
    signature = infer_signature(model_input=X_train, model_output=model.predict(X_train))

    # mlflow: log metrics
    log_mlflow_run(model, signature)

    # mlflow: end tracking
    mlflow.end_run()
    
    progress_bar.value = i + 1
    
    # Update progress text
    with progress_text:
        print(f"Learning rate: {run_parameters['learning_rate']}"
              f"\tMax depth: {run_parameters['max_depth']}")
        print(message)
        print(message_cv)
        print()

## Best Model Results

After training several models and tuning their hyperparameters, we will identify the model that performs the best according to our chosen evaluation metrics.

In [ ]:
best_run_df = mlflow.search_runs(order_by=['metrics.RMSLE_CV ASC'], max_results=1)
if len(best_run_df.index) == 0:
    raise Exception(f"Found no runs for experiment '{experiment_name}'")

best_run = mlflow.get_run(best_run_df.at[0, 'run_id'])
best_model_uri = f"{best_run.info.artifact_uri}/model"
best_model = mlflow.sklearn.load_model(best_model_uri)

In [ ]:
# Print best run info
print("Best run info:")
print(f"Run id: {best_run.info.run_id}")
print(f"Run parameters: {best_run.data.params}")
print(f"Run score: RMSLE_CV = {best_run.data.metrics['RMSLE_CV']:.4f}")
print(f"Run model URI: {best_model_uri}")

In [ ]:
model_feature_importance(best_model)

In [ ]:
model_permutation_importance(best_model)

In [ ]:
model_tree_visualization(best_model)

## Model Testing

Once we have identified our best model, the next step is to test its predictive performance on unseen data. This is done using the test dataset, which has been set aside specifically for this purpose.

Testing the model's predictions allows us to evaluate how well our model generalizes to new data. This is a crucial step in the machine learning process, as it provides a realistic estimate of the model's performance in a real-world setting.

We will compare the model's predictions with the actual values in the test dataset and calculate our chosen evaluation metrics. These results will give us a clear indication of the model's predictive accuracy.

In [ ]:
test_predictions = X_test.copy()
# real output (rented_bikes) from test dataset
test_predictions["rented_bikes"] = y_test

# add "predicted_rented_bikes" from test dataset
test_predictions["predicted_rented_bikes"] = best_model.predict(X_test).astype(int)

# show results
test_predictions.head()

In [ ]:
# plot truth vs prediction values
test_predictions.plot(
    kind="scatter",
    x="rented_bikes",
    y="predicted_rented_bikes",
    title="Rented bikes vs predicted rented bikes",
    figsize=(10, 10)
)

plt.show()

## Model Deployment

In this section of the notebook, you will focus on deploying the trained model
and bridge the gap between insightful data analysis and tangible real-world impact.
For this, you will be using KServe, an open-source platform that facilitates the deployment
and management of machine learning models at scale. It provides a robust and scalable
infrastructure to serve predictions from trained models in production environments. The
backend that you'll be using for KServe is Seldon.

Before diving into deployment, you should start by creating a secure environment for accessing the S3 endpoint. First, you define a Secret object, which securely holds the necessary credentials. Additionally, we create a ServiceAccount object, associating it with the secret to establish an identity for the deployment process.

In [ ]:
# Access Key and Secret Key inputs
namespace_input = widgets.Text(description='Namespace:')
submit_button = widgets.Button(description='Submit')
success_message = widgets.Output()

namespace = None

def submit_button_clicked(b):
    global namespace
    namespace = namespace_input.value
    with success_message:
        success_message.clear_output()
        print("Namespace submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)

# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

# Display inputs and button
display(namespace_input, submit_button, success_message)

In [ ]:
k8s_client = client.CoreV1Api()

minio_secret = V1Secret(
    api_version="v1",
    kind="Secret",
    metadata=V1ObjectMeta(
        name="kserve-minio-secret",
        namespace=namespace,
        annotations={
            "serving.kserve.io/s3-endpoint": f"home.{EZAF_ENV}.com:31900",
            "serving.kserve.io/s3-usehttps": "1",
            "serving.kserve.io/s3-verifyssl": "0",
            "serving.kserve.io/s3-useanoncredential": "false",
            "serving.kserve.io/s3-cabundle": ""}),
    type="Opaque",
    string_data={
        "AWS_ACCESS_KEY_ID": minio_access_key,
        "AWS_SECRET_ACCESS_KEY": minio_secret_key})

k8s_client.create_namespaced_secret(namespace, minio_secret)

In [ ]:
sa = V1ServiceAccount(
    api_version="v1",
    kind="ServiceAccount",
    metadata=V1ObjectMeta(name="kserve-minio-sa"),
    secrets=[V1ObjectReference(
        api_version="v1",
        kind="Secret",
        name="kserve-minio-secret",
        namespace=namespace)])

k8s_client.create_namespaced_service_account(namespace, sa)

Finally, you can configure your InferenceService (ISVC) object to create your deployment. Once the ISVC is ready, launch the `bike-sharing-prediction.ipynb` notebook to send your first request to your model's endpoint.

In [ ]:
isvc = V1beta1InferenceService(
    api_version="serving.kserve.io/v1beta1",
    kind="InferenceService",
    metadata=V1ObjectMeta(name="bike-sharing"),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            service_account_name="kserve-minio-sa",
            sklearn=V1beta1SKLearnSpec(
                storage_uri=best_model_uri,
                protocol_version="v2"))))

kserve_client = kserve.KServeClient()
kserve_client.create(isvc, namespace=namespace)